In [1]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
import numpy as np

model = VGG16(weights='imagenet')

img_path = 'lion.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
print('Predicted:', decode_predictions(preds, top=3)[0])

Predicted: [('n02129165', 'lion', 0.9999999), ('n02130308', 'cheetah', 7.703386e-08), ('n02128385', 'leopard', 6.330456e-09)]


In [2]:
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [3]:
model.save('vgg16/1')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: vgg16/1/assets


# Running the server 

**tensorflow_model_server --rest_api_port=8000 --model_config_file=models.config --model_config_file_poll_wait_seconds=300**

In [4]:
import json
import requests

In [5]:
data = json.dumps({"instances": x.tolist()})

headers = {"content-type": "application/json"}

json_response = requests.post('http://localhost:8000/v1/models/vgg16/versions/1:predict',
                              data=data,
                              headers=headers)

predictions = json.loads(json_response.text)
print('Predicted:', decode_predictions(preds, top=3)[0])

Predicted: [('n02129165', 'lion', 0.9999999), ('n02130308', 'cheetah', 7.703386e-08), ('n02128385', 'leopard', 6.330456e-09)]


# Serving with Docker

docker pull tensorflow/serving

docker run -p 8501:8501 --name tf_vgg_server --mount type=bind,source=/media/derrick/5EAD61BA2C09C31B/Notebooks/Python/serving/saved_tf_model,target=/models/vgg16  -e MODEL_NAME=vgg16 -t tensorflow/serving &


In [8]:
json_response = requests.post('http://127.0.0.1:8501/v1/models/vgg16:predict',
                              data=data,
                              headers=headers)

predictions = json.loads(json_response.text)
print('Predicted:', decode_predictions(preds, top=3)[0])

Predicted: [('n02129165', 'lion', 0.9999999), ('n02130308', 'cheetah', 7.703386e-08), ('n02128385', 'leopard', 6.330456e-09)]
